In [1]:
import cv2
import numpy as np
import os
import math
import json

In [2]:
# Contour parameters
min_contour_width = 50  
min_contour_height = 50
max_contour_width = 300 
max_contour_height = 300
# Distance from the line
offset = 15
# Lane parameters
lane_1_height_1=283
lane_1_height_2=283
lane_1_width_1=270
lane_1_width_2=1280
matches = []
# Vehicle counters
vehicles_3 = 0
vehicles_2 = 0
# Other helping metrics
frame_number_threshold=16

In [3]:
# Getting centroid
def get_centrolid(x, y, w, h):
   x1 = int(w / 2)
   y1 = int(h / 2)
 
   cx = x + x1
   cy = y + y1
   return cx, cy

In [4]:
# Loading videos
files=[]
folder_path = os.getcwd()
search_string="Sanity"
for filename in os.listdir(folder_path):
    if search_string in filename:
        files.append(filename)

In [5]:
print(files)

['SanityCheck.mp4']


In [ ]:
results={}

last_1=""

# Iterating through videos

for video in files[:]:

    # Defining counters

    vehicles_1 = 0

    motorbikes_1=0

    cars_1=0

    transporters_1=0

    trucks_1=0

    motorbikes_1_list=[]

    cars_1_list=[]

    transporters_1_list=[]

    trucks_1_list=[]

    # Loading videos
    
    cap = cv2.VideoCapture(video)
    
    #cap.set(3, 620)
    #cap.set(4, 480)
     
    if cap.isOpened():
       ret, frame1 = cap.read()
    else:
       ret = False
    ret, frame1 = cap.read()
    ret, frame2 = cap.read()
    
    frame_nr=0
    frame_1_counter=0
    
    while ret:
    
       frame_1_counter+=1

       # Frame difference calculations and preprocessing
    
       d = cv2.absdiff(frame1, frame2)
       grey = cv2.cvtColor(d, cv2.COLOR_BGR2GRAY)
     
       blur = cv2.GaussianBlur(grey, (5, 5), 0)
     
       ret, th = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
       dilated = cv2.dilate(th, np.ones((3, 3)))
       kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
     
      
       closing = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, kernel)
       contours, h = cv2.findContours(
           closing, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
       for(i, c) in enumerate(contours):
           (x, y, w, h) = cv2.boundingRect(c)
           contour_valid = (w >= min_contour_width) and (h >= min_contour_height) and (h <= max_contour_height) and (w <= max_contour_width)
     
           if not contour_valid:
               continue

           # Drawing rectangles over centroids
           
           cv2.rectangle(frame1, (x-10, y-10), (x+w+10, y+h+10), (255, 0, 0), 2)

           # Drawing lines
     
           cv2.line(frame1, (lane_1_width_1, lane_1_height_1), (lane_1_width_2, lane_1_height_2), (0, 255, 0), 5)
           centrolid = get_centrolid(x, y, w, h)
           matches.append(centrolid)
           cv2.circle(frame1, centrolid, 5, (0, 255, 0), -1)
           cx, cy = get_centrolid(x, y, w, h)

           # Checking if each point in within the given distance from the line, on right width and classifying vehicle type
           
           for (x, y) in matches:
               if x > lane_1_width_1 and x <lane_1_width_2 and abs(y-lane_1_height_2)<offset:
                    if frame_1_counter<=frame_number_threshold+15 and frame_1_counter>25:
                       if last_1=="motorbike":
                           motorbikes_1-=1
                           try:
                               motorbikes_1_list.pop()
                           except:
                               pass
                       if last_1=="car":
                           cars_1-=1
                           try:
                               cars_1_list.pop()
                           except:
                               pass
                       if last_1=="transporter":
                           transporters_1-=1
                           try:
                               transporters_1_list.pop()
                           except:
                               pass
                       if last_1=="truck":
                           trucks_1-=1
                           try:
                               trucks_1_list.pop()
                           except:
                               pass
                       trucks_1+=1
                       trucks_1_list.append(frame_nr)
                       frame_1_counter=0
                    elif frame_1_counter<=25 and frame_1_counter>20:
                       if last_1=="motorbike":
                           motorbikes_1-=1
                           try:
                               motorbikes_1_list.pop()
                           except:
                               pass
                       if last_1=="car":
                           cars_1-=1
                           try:
                               cars_1_list.pop()
                           except:
                               pass
                       if last_1=="transporter":
                           transporters_1-=1
                           try:
                               transporters_1_list.pop()
                           except:
                               pass
                       if last_1=="truck":
                           trucks_1-=1
                           try:
                               trucks_1_list.pop()
                           except:
                               pass
                       transporters_1+=1
                       transporters_1_list.append(frame_nr)
                       frame_1_counter=0
                    elif frame_1_counter>frame_number_threshold:
                       if w<=60 and h<=60:
                           motorbikes_1+=1
                           motorbikes_1_list.append(frame_nr)
                           last_1="motorbike"
                       elif w>200 and h>200 and w<200 and h<200:
                           cars_1+=1
                           cars_1_list.append(frame_nr)
                           last_1="car"
                       elif w<200 and h<200 and w>=250 and h>=250:
                           transporters_1+=1
                           transporters_1_list.append(frame_nr)
                           last_1="transporter"
                       elif w>=250 and h>=250:
                           trucks_1+=1
                           trucks_1_list.append(frame_nr)
                           last_1="truck"
                       else:
                           cars_1+=1
                           cars_1_list.append(frame_nr)
                           last_1="car"
                       frame_1_counter=0
                       vehicles_1+=1
                    try:
                       matches.remove((x, y))
                    except:
                       pass

       # Text with current counts on the video
     
       cv2.putText(frame1, f"Total Vehicle Detected Lane 1: {vehicles_1}", (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                   (0, 170, 0), 2)
       cv2.putText(frame1, f"Motorbikes: {motorbikes_1}, cars: {cars_1}, transporters: {transporters_1}, trucks: {trucks_1}", (0, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.4,
                   (170, 0, 0), 1)
        
       if cv2.waitKey(1) == 27:
           break
       # Wait for a specified amount of time
       if cv2.waitKey(1) & 0xFF == ord('q'):
            break
       frame1 = frame2
       ret, frame2 = cap.read()
       frame_nr+=1

       #print(frame_nr)
     
    cv2.destroyAllWindows()
    cap.release()

    # Saving results into dictionary

    results[video]={"motorbikes_lane_1":motorbikes_1, "motorbikes_lane_1_frames":motorbikes_1_list, "cars_lane_1":cars_1, "cars_lane_1_frames":cars_1_list, "transporters_lane_1":transporters_1, "transporters_lane_1_frames":transporters_1_list, "trucks_lane_1":trucks_1, "trucks_lane_1_frames":trucks_1_list}

In [ ]:
# Saving into json file

with open(f"{search_string}.json", "w") as json_file:
    json.dump(results, json_file)